# Download Sentinel 1 RTC Data

Import necessary libraries

In [ ]:
# imports
# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

# Visualization
import ipyleaflet
import matplotlib.pyplot as plt
from IPython.display import Image
import seaborn as sns

# Data Science
import numpy as np
import pandas as pd
import xarray as xr

# Planetary Computer Tools
import pystac
import pystac_client
import odc
from pystac_client import Client
from pystac.extensions.eo import EOExtension as eo
from odc.stac import stac_load
import planetary_computer as pc
pc.settings.set_subscription_key('cf5657d28bb2408ba8fd775642c2e1cb')

# Others
import requests
import rich.table
from itertools import cycle
from tqdm import tqdm
from datetime import datetime
import os
from glob import glob

## Downloading Annual Data and Variety of Pixel Areas

<p align="justify">For this experiment the author chose to download data between <b>1st January 2022</b> until <b>31th December 2022</b>. The author also experiment using various of pixel area start from:
    <ol>
        <li><b>1x1</b>,</li>
        <li><b>3x3</b>,</li>
        <li><b>5x5</b>,</li>
        <li><b>7x7</b>, and</li>
        <li><b>10x10</b></li>
    </ol>
</p>

## 1. Preparing Train and Test Coordinates

In [ ]:
# reading train data
train_data = pd.read_csv("../../data/Crop_Location_Data.csv")
display(train_data)

# reading test data
test_data = pd.read_csv('../../data/challenge_1_submission_template.csv')
test_data = test_data.rename(columns={"id": "Latitude and Longitude"})
display(test_data)

## 2. Prepare Box Size Area

In [ ]:
# prepare bounding box
box_size_deg_3x3 = 0.0002 # Surrounding box in degrees, yields approximately 3x3 pixel region
box_size_deg_5x5 = 0.0004 # Surrounding box in degrees, yields approximately 5x5 pixel region
box_size_deg_7x7 = 0.0006 # Surrounding box in degrees, yields approximately 7x7 pixel region
box_size_deg_10x10 = 0.0009 # Surrounding box in degrees, yields approximately 10x10 pixel region

def create_bbox(lat_long, box_size_deg):
    lat, lon = float(lat_long[0]), float(lat_long[1])
    min_lon = lon-box_size_deg/2
    min_lat = lat-box_size_deg/2
    max_lon = lon+box_size_deg/2
    max_lat = lat+box_size_deg/2
    return (min_lon, min_lat, max_lon, max_lat)

## 3. Prepare Function for Getting the Sentinel 1 RTC Data

In [ ]:
def get_sentinel_data(latlong,time_slice,assets, box_size_deg=None):
    '''
    Returns VV and VH values for a given latitude and longitude 
    Attributes:
    latlong - A tuple with 2 elements - latitude and longitude
    time_slice - Timeframe for which the VV and VH values have to be extracted
    assets - A list of bands to be extracted
    '''
    # 1. extract the longitude and latitude from string
    latlong=latlong.replace('(','').replace(')','').replace(' ','').split(',')
    
    # 2. if the box_size_deg is none then the pixel area will be set to 1x1
    if box_size_deg is None:
        bbox_of_interest = (float(latlong[1]) , float(latlong[0]), float(latlong[1]) , float(latlong[0]))
    else:
        bbox_of_interest = create_bbox(latlong, box_size_deg)
    time_of_interest = time_slice
    
    # 3. access the Azure Blob Storage that holding the data
    catalog = pystac_client.Client.open(
        "https://planetarycomputer.microsoft.com/api/stac/v1"
    )
    
    # 4. search the data
    search = catalog.search(
        collections=["sentinel-1-rtc"], bbox=bbox_of_interest, datetime=time_of_interest
    )
    items = list(search.get_all_items())
    
    # 5. load the data bands
    bands_of_interest = assests
    data = stac_load(items, patch_url=pc.sign, bbox=bbox_of_interest)
    return data

## 4. Download the Data

In [ ]:
# set up time slice and assets
time_slice = "2022-01-01/2022-12-31"
assests = ['vh','vv']

### 1x1

#### Train Data

In [ ]:
# Prepare the Folder
directory="../../data/train/sentinel-1-rtc/1-year-202201-202212/vh-vv-1x1"
if not os.path.exists(directory):
    print("Creating %s" % (directory))
    os.makedirs(directory)
    print("Done")
else:
    print("%s already existed" % (directory))

# Download the Data
for coordinates in tqdm(train_data['Latitude and Longitude']):
    path = os.path.join(directory, coordinates+".nc")
    if not os.path.exists(path):
        data = get_sentinel_data(coordinates,time_slice,assests)
        data.to_netcdf(path)

#### Test Data

In [ ]:
# Prepare the Folder
directory="../../data/test/sentinel-1-rtc/1-year-202201-202212/vh-vv-1x1"
if not os.path.exists(directory):
    print("Creating %s" % (directory))
    os.makedirs(directory)
    print("Done")
else:
    print("%s already existed" % (directory))

# Download the Data
for coordinates in tqdm(test_data['Latitude and Longitude']):
    path = os.path.join(directory, coordinates+".nc")
    if not os.path.exists(path):
        data = get_sentinel_data(coordinates,time_slice,assests)
        data.to_netcdf(path)

### 3x3

#### Train Data

In [ ]:
# Prepare the Folder
directory="../../data/train/sentinel-1-rtc/1-year-202201-202212/vh-vv-3x3"
if not os.path.exists(directory):
    print("Creating %s" % (directory))
    os.makedirs(directory)
    print("Done")
else:
    print("%s already existed" % (directory))

# Download the Data
for coordinates in tqdm(train_data['Latitude and Longitude']):
    path = os.path.join(directory, coordinates+".nc")
    if not os.path.exists(path):
        data = get_sentinel_data(coordinates,time_slice,assests, box_size_deg_3x3)
        data.to_netcdf(path)

#### Test Data

In [ ]:
# Prepare the Folder
directory="../../data/test/sentinel-1-rtc/1-year-202201-202212/vh-vv-3x3"
if not os.path.exists(directory):
    print("Creating %s" % (directory))
    os.makedirs(directory)
    print("Done")
else:
    print("%s already existed" % (directory))

# Download the Data
for coordinates in tqdm(test_data['Latitude and Longitude']):
    path = os.path.join(directory, coordinates+".nc")
    if not os.path.exists(path):
        data = get_sentinel_data(coordinates,time_slice,assests, box_size_deg_3x3)
        data.to_netcdf(path)

### 5x5

#### Train Data

In [ ]:
# Prepare the Folder
directory="../../data/train/sentinel-1-rtc/1-year-202201-202212/vh-vv-5x5"
if not os.path.exists(directory):
    print("Creating %s" % (directory))
    os.makedirs(directory)
    print("Done")
else:
    print("%s already existed" % (directory))

# Download the Data
for coordinates in tqdm(train_data['Latitude and Longitude']):
    data = get_sentinel_data(coordinates,time_slice,assests, box_size_deg_5x5)
    data.to_netcdf(os.path.join(directory, coordinates+".nc"))

#### Test Data

In [ ]:
# Prepare the Folder
directory="../../data/test/sentinel-1-rtc/1-year-202201-202212/vh-vv-5x5"
if not os.path.exists(directory):
    print("Creating %s" % (directory))
    os.makedirs(directory)
    print("Done")
else:
    print("%s already existed" % (directory))

# Download the Data
for coordinates in tqdm(test_data['Latitude and Longitude']):
    path = os.path.join(directory, coordinates+".nc")
    if not os.path.exists(path):
        data = get_sentinel_data(coordinates,time_slice,assests, box_size_deg_5x5)
        data.to_netcdf(path)

### 7x7

#### Train Data

In [ ]:
# Prepare the Folder
directory="../../data/train/sentinel-1-rtc/1-year-202201-202212/vh-vv-7x7"
if not os.path.exists(directory):
    print("Creating %s" % (directory))
    os.makedirs(directory)
    print("Done")
else:
    print("%s already existed" % (directory))

# Download the Data
for coordinates in tqdm(train_data['Latitude and Longitude']):
    path = os.path.join(directory, coordinates+".nc")
    if not os.path.exists(path):
        data = get_sentinel_data(coordinates,time_slice,assests, box_size_deg_7x7)
        data.to_netcdf(path)

#### Test Data

In [ ]:
# Prepare the Folder
directory="../../data/test/sentinel-1-rtc/1-year-202201-202212/vh-vv-7x7"
if not os.path.exists(directory):
    print("Creating %s" % (directory))
    os.makedirs(directory)
    print("Done")
else:
    print("%s already existed" % (directory))

# Download the Data
for coordinates in tqdm(test_data['Latitude and Longitude']):
    path = os.path.join(directory, coordinates+".nc")
    if not os.path.exists(path):
        data = get_sentinel_data(coordinates,time_slice,assests, box_size_deg_7x7)
        data.to_netcdf(path)

### 10x10

#### Train Data

In [ ]:
# Prepare the Folder
directory="../../data/train/sentinel-1-rtc/1-year-202201-202212/vh-vv-10x10"
if not os.path.exists(directory):
    print("Creating %s" % (directory))
    os.makedirs(directory)
    print("Done")
else:
    print("%s already existed" % (directory))

# Download the Data
for coordinates in tqdm(train_data['Latitude and Longitude']):
    path = os.path.join(directory, coordinates+".nc")
    if not os.path.exists(path):
        data = get_sentinel_data(coordinates,time_slice,assests, box_size_deg_10x10)
        data.to_netcdf(path)

#### Test Data

In [ ]:
# Prepare the Folder
directory="../../data/test/sentinel-1-rtc/1-year-202201-202212/vh-vv-10x10"
if not os.path.exists(directory):
    print("Creating %s" % (directory))
    os.makedirs(directory)
    print("Done")
else:
    print("%s already existed" % (directory))

# Download the Data
for coordinates in tqdm(test_data['Latitude and Longitude']):
    path = os.path.join(directory, coordinates+".nc")
    if not os.path.exists(path):
        data = get_sentinel_data(coordinates,time_slice,assests, box_size_deg_10x10)
        data.to_netcdf(path)